# Multiple-Model Disease Prediction

**Purpose:** Use seven different models and a voting classifier to predict three different diseases and compare their performance. Will also try different pre-processing methods to optimize each model.

## Outline

1. Load and pre-process admissions data
2. Analyze common diseases for model evaluation
3. Build pipeline for each disease
    1. Sub-sample list of patients and lab events
    3. Load and pre-process lab data
    4. Calculate aggregate test values
    5. Transform and process data for modeling
    6. Generate models
4. Store models in object to evaluate

## Imports

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

from sklearn.utils import resample
# from sklearn.impute import KNNImputer
# from sklearn.preprocessing import StandardScaler
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import NearMiss

# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve
# from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, \
#                             recall_score, f1_score, roc_auc_score, plot_roc_curve

## 1. Load and pre-process admissions data

In [2]:
%%time
path = "D:\\Bootcamp\\MIMIC IV\\"
admissions = pd.read_csv(path + "core\\admissions.csv.gz", compression='gzip')
patients = pd.read_csv(path + "core\\patients.csv.gz", compression='gzip')
# transfers = pd.read_csv(path + "core\\transfers.csv.gz", compression='gzip')
diagnoses = pd.read_csv(path + "hosp\\diagnoses_icd.csv.gz", compression='gzip')
d_diagnoses = pd.read_csv(path + "hosp\\d_icd_diagnoses.csv.gz", compression='gzip')
drg_codes = pd.read_csv(path + "hosp\\drgcodes.csv.gz", compression='gzip')
d_lab_events = pd.read_csv(path + "hosp\\d_labitems.csv.gz", compression='gzip')

Wall time: 9.71 s


In [3]:
%%time
# Basic pre-processing.
time_cols = ['admittime', 'dischtime', 'edregtime', 'edouttime']
text_cols = ['admission_type', 'admission_location', 'discharge_location',
              'insurance', 'language', 'marital_status', 'ethnicity']
admissions[text_cols] = admissions[text_cols].apply(lambda x: x.str.lower())
admissions[time_cols] = admissions[time_cols].astype('datetime64')
diagnoses['icd_code'] = diagnoses['icd_code'].str.strip()
d_diagnoses['icd_code'] = d_diagnoses['icd_code'].str.strip()
d_diagnoses['long_title'] = d_diagnoses['long_title'].str.lower()
drg_codes['description'] = drg_codes['description'].str.lower()

# Remove duplicate drg codes and merge with admissions.
drg_codes = drg_codes[drg_codes.drg_type=='APR'].groupby('hadm_id').first().reset_index()
admissions = admissions.merge(drg_codes[['subject_id', 'hadm_id', 'description']],
                              how='left', on=['subject_id', 'hadm_id'])

# Add icd code descriptions to diagnoses table and fill missing codes.
diagnoses = diagnoses.merge(d_diagnoses, how='left', on=['icd_version', 'icd_code'])
mis_diag = diagnoses[diagnoses.long_title.isna()].drop(columns='long_title').copy()
mis_diag['icd_code'] = mis_diag['icd_code'] + '1'
mis_diag = mis_diag.merge(d_diagnoses, how='left', on=['icd_version', 'icd_code'])
mis_diag['long_title'] = mis_diag['long_title'].fillna('Bad ICD Code')
diagnoses = diagnoses.append(mis_diag).reset_index(drop=True)
diagnoses = diagnoses.drop_duplicates(subset=['subject_id', 'hadm_id', 'seq_num'], keep='last')

# Flatten icd codes to single cell and merge with admissions table.
diagnoses_flat = diagnoses.groupby('hadm_id').agg({'long_title': lambda x: '; '.join(x)})
admissions = admissions.merge(diagnoses_flat, how='left', on='hadm_id')
admissions = admissions.rename(columns={'description':'drg_code', 'long_title':'icd_code'})
admissions[['drg_code', 'icd_code']] = admissions[['drg_code', 'icd_code']].fillna('No code recorded')

Wall time: 20.9 s


## 2. Analyze common diseases for model evaluation

In [4]:
diagnoses[diagnoses.icd_version==9].long_title.value_counts().head(50) // 2508.340 / 10

unspecified essential hypertension                                                                                        3.2
other and unspecified hyperlipidemia                                                                                      2.2
esophageal reflux                                                                                                         1.5
diabetes mellitus without mention of complication, type ii or unspecified type, not stated as uncontrolled                1.3
depressive disorder, not elsewhere classified                                                                             1.2
atrial fibrillation                                                                                                       1.1
personal history of tobacco use                                                                                           1.1
congestive heart failure, unspecified                                                                                 

In [5]:
drg_codes.description.value_counts().head(50) // (drg_codes.shape[0]/1000) / 10

neonate, bwt > 2499g, normal newborn or neonate w other problem       13.1
vaginal delivery                                                       2.6
septicemia & disseminated infections                                   2.4
heart failure                                                          2.3
other pneumonia                                                        1.4
cesarean delivery                                                      1.3
renal failure                                                          1.3
chemotherapy                                                           1.3
percutaneous cardiovascular procedures w/o ami                         1.1
kidney & urinary tract infections                                      1.1
other vascular procedures                                              1.1
other digestive system diagnoses                                       1.1
major small & large bowel procedures                                   1.1
cellulitis & other bacter

#### Candidates to research:

* heart failure
* kidney failure
* heart & kidney failure
* acute myocardial infarction (heart attack)
* COPD
* pancreas disorders
* sepsis
* psychoses
* major depressive


In [6]:
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,drg_code,icd_code
0,12427812,21593330,2184-01-06 11:51:00,2184-01-10 11:45:00,NaN,urgent,physician referral,home,other,english,NaN,unknown,NaT,NaT,0,cesarean delivery,single live birth; maternal care for low trans...
1,14029832,22059088,2120-01-18 01:28:00,2120-01-20 16:13:00,NaN,urgent,transfer from hospital,home,other,english,NaN,other,NaT,NaT,0,diverticulitis & diverticulosis,type 2 diabetes mellitus with hyperglycemia; b...
2,14495017,22484010,2175-01-28 15:41:00,2175-01-29 16:00:00,NaN,direct emer.,physician referral,home,other,?,NaN,white,NaT,NaT,0,"neonate, bwt > 2499g, normal newborn or neonat...",other neonatal jaundice due to delayed conjuga...
3,13676048,23865469,2193-01-19 05:27:00,2193-01-24 18:59:00,NaN,urgent,physician referral,home,other,?,married,white,NaT,NaT,0,cesarean delivery,single live birth; maternal care for excessive...
4,13831972,27763544,2131-01-27 04:03:00,2131-01-27 05:39:00,NaN,eu observation,emergency room,NaN,medicaid,english,single,white,2131-01-26 22:19:00,2131-01-27 05:39:00,0,No code recorded,"psychophysical visual disturbances; migraine, ..."


In [7]:
patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10002723,F,0,2128,2017 - 2019,NaN
1,10003939,M,0,2184,2008 - 2010,NaN
2,10004222,M,0,2161,2014 - 2016,NaN
3,10005325,F,0,2154,2011 - 2013,NaN
4,10007338,F,0,2153,2017 - 2019,NaN


In [9]:
def research_disease(keyword, exclusion='NA'):
    if type(keyword) == str:
        hadm_list = admissions[admissions.icd_code.str.contains(keyword) &
                               ~admissions.icd_code.str.contains(exclusion)].hadm_id
    else:
        hadm_list = admissions[admissions.icd_code.str.contains(keyword[0]) &
                               admissions.icd_code.str.contains(keyword[1])].hadm_id
    disease_admissions = admissions[admissions.hadm_id.isin(hadm_list)].copy()
    disease_patients = patients[patients.subject_id.isin(disease_admissions.subject_id.unique())].copy()
    
    print('='*50)
    print(f"Disease: \033[1m{keyword}\033[0m")
    print('-'*40)
    print(f"\t(excluding: {exclusion})")
    print("\nEpidemiology Information\n", '-'*40)
    print(f"Prevalence: {len(disease_admissions)/len(admissions):.2%} of visits")
    print(f"            {len(disease_patients)/len(patients):.2%} of patients")
    print(f"Death rate: {disease_admissions.hospital_expire_flag.mean():.2%}")
    avg_visits = np.mean(admissions[admissions.subject_id.isin(disease_patients.subject_id)]
                         .groupby('subject_id').hadm_id.count())
    print(f"Avg number of visits for patients who develop this condition: {avg_visits:.1f}")
    avg_pos_visits = len(disease_admissions)/len(disease_patients)
    percent_positive = avg_pos_visits/avg_visits
    print(f"Percentage of visits where condition is present: {percent_positive:.2%}")
    print("\nDemographic Information\n", '-'*40)
    print(f"Average age: {disease_patients.anchor_age.mean():.1f} yr")
    print(f"\nGender breakdown: \n{disease_patients.gender.value_counts()/len(disease_patients)//.001/10}")
    print(f"\nEthnicity breakdown: \n{disease_admissions.ethnicity.value_counts()/len(disease_admissions)//.001/10}")
    print('='*50,'\n')

In [10]:
research_disease('heart failure', 'kidney')
research_disease('kidney failure', 'heart failure')
research_disease(['kidney failure', 'heart failure'])
research_disease('acute myocardial infarction')
research_disease('chronic obstructive pulmonary disease')
research_disease('pancreas')
research_disease('septicemia')
research_disease('psychosis', 'major depressive')
research_disease('major depressive')

Disease: heart failure
----------------------------------------
	(excluding: kidney)

Epidemiology Information
 ----------------------------------------
Prevalence: 4.33% of visits
            3.27% of patients
Death rate: 2.36%
Avg number of visits for patients who develop this condition: 4.7
Percentage of visits where condition is present: 38.33%

Demographic Information
 ----------------------------------------
Average age: 70.9 yr

Gender breakdown: 
F    50.9
M    49.0
Name: gender, dtype: float64

Ethnicity breakdown: 
white                            72.9
black/african american           13.0
hispanic/latino                   4.5
other                             3.7
unknown                           3.1
asian                             1.9
unable to obtain                  0.4
american indian/alaska native     0.1
Name: ethnicity, dtype: float64

Disease: kidney failure
----------------------------------------
	(excluding: heart failure)

Epidemiology Information
 ------------

# 3. Build a Pipeline for Each Disease

## 3.A. Sub-Sample List of Patients and Lab Events

In [28]:
# Sample patients, stratify by positive admissions.
include = 'heart failure'
exclude = 'kidney'
n_samples = 1000
pos_hadm = admissions[admissions.icd_code.str.contains(include) &
                               ~admissions.icd_code.str.contains(exclude)].hadm_id
admissions['pos_hadm'] = np.where(admissions.hadm_id.isin(pos_hadm), 1, 0)
print(f"Base positivity rate: {admissions.pos_hadm.mean():.2%}")
sample_admissions = resample(admissions[['subject_id', 'hadm_id', 'pos_hadm']], \
                       n_samples = n_samples, replace = False, \
                       stratify = admissions.pos_hadm, random_state = 0)
sample_admissions = sample_admissions.drop_duplicates('subject_id', keep='last')
print(f"{n_samples - len(sample_admissions)} repeat visits were dropped.")
sample_hadm = sample_admissions.hadm_id
pos_hadm = sample_admissions[sample_admissions.pos_hadm==1].hadm_id
print(f"Sample positivity rate: {len(pos_hadm)/len(sample_hadm):.2%}")
n_samples = len(sample_admissions)

Base positivity rate: 4.33%
2 repeat visits were dropped.
Sample positivity rate: 4.21%


In [12]:
print(sample_admissions.shape)
sample_admissions.head()

(998, 3)


,subject_id,hadm_id,pos_hadm
453041,12541102,26940632,0
206495,11670510,23132542,0
244459,11721838,27196834,0
365158,18289606,27576102,0
37992,13895041,24913772,1


In [17]:
# %%time

# i = 1
# labevent_cols = ['subject_id', 'hadm_id', 'specimen_id', 'itemid', 'value', 'valuenum', 'valueuom', \
#          'ref_range_lower', 'ref_range_upper', 'comments', 'flag']
# lab_events = []
# for chunk in pd.read_csv(path + "hosp\labevents.csv.gz", compression='gzip', chunksize=3e6):
#     clear_output()
#     chunk = chunk[chunk.hadm_id.isin(sample_hadm)]
#     chunk = chunk[labevent_cols]
#     lab_events.append(chunk)
#     print(f'Processed chunk: {i}')
#     i += 1
# clear_output()
# print('Processing complete!')
# lab_events = pd.concat(lab_events)

## 3.B. Load and pre-process lab data

In [566]:
lab_events = pd.read_pickle("data\lab_events.pkl")
lab_events.hadm_id = lab_events.hadm_id.astype(int)
lab_events.value = np.where(lab_events.value.isna(), lab_events.value, lab_events.value.astype(str))
lab_events.valuenum = lab_events.valuenum.astype(float)
labs = lab_events
print(lab_events.shape)
lab_events.sample(10)

(130270, 11)


,subject_id,hadm_id,specimen_id,itemid,value,valuenum,valueuom,ref_range_lower,ref_range_upper,comments,flag
37963,15952403,24636286,93523336,51277,16.2,16.2,%,10.5,15.5,NaN,abnormal
110525,10952380,21635695,13144441,50882,17,17.0,mEq/L,22.0,32.0,NaN,abnormal
109667,14560913,29832680,96489525,51266,NaN,NaN,NaN,NaN,NaN,VERY HIGH.,NaN
18432,13822273,24500702,834014,51248,28.3,28.3,pg,27.0,32.0,NaN,NaN
47604,18965721,25198149,20310516,51487,NaN,NaN,NaN,NaN,NaN,NEG.,NaN
67489,14161837,28023470,73699755,51248,29.9,29.9,pg,26.0,32.0,NaN,NaN
108199,14560913,29832680,96921252,51484,NaN,NaN,mg/dL,NaN,NaN,NEG.,NaN
25467,19798925,21828100,72224792,51265,204,204.0,K/uL,150.0,400.0,NaN,NaN
36113,14186271,24277490,98629092,50960,2.1,2.1,mg/dL,1.6,2.6,NaN,NaN
68391,15751968,22660664,60235413,51006,15,15.0,mg/dL,6.0,20.0,NaN,NaN


In [567]:
print(f"Fraction of sample patients with lab events: {lab_events.hadm_id.nunique()/n_samples:.1%}")
print(f"Number of unique tests: {lab_events.itemid.nunique()}")
print(f"Average number of specimens per patient: {len(lab_events)/lab_events.specimen_id.nunique():.1f}")
print(f"Average number of tests per specimen: {lab_events.specimen_id.nunique()/lab_events.hadm_id.nunique():.1f}")

Fraction of sample patients with lab events: 81.4%
Number of unique tests: 519
Average number of specimens per patient: 9.5
Average number of tests per specimen: 16.8


In [568]:
# Include only the most common lab tests in the model.

def find_common_tests(df, threshold):
    test_frequency = df.groupby('itemid').subject_id.nunique()/df.subject_id.nunique()
    common_tests = test_frequency[test_frequency >= threshold]
    return common_tests.index.tolist()

threshold = 0.5
test_list = []
test_list.append(find_common_tests(labs[labs.hadm_id.isin(pos_hadm)], threshold))
test_list.append(find_common_tests(labs[~labs.hadm_id.isin(pos_hadm)], threshold))
test_list = list(dict.fromkeys(sum(test_list, [])))   # Remove duplicates.

lab_events = lab_events[lab_events.itemid.isin(test_list)]

In [570]:
# Remove junk data that provides no information
lab_events = lab_events[~(lab_events.value.isna() & lab_events.valuenum.isna() & lab_events.comments.isna())]
lab_events = lab_events[~(lab_events.value.isna() & lab_events.valuenum.isna() & (lab_events.comments=='___'))]

# Remove rows with errors
lab_events = lab_events[~lab_events.value.fillna('').str.contains('ERROR|UNABLE')]
lab_events = lab_events[~lab_events.comments.fillna('').str.contains('ERROR|UNABLE')]

# Check for rows where value != valuenum
lab_events[(lab_events.value.str.replace('<|>', '').astype(float) - lab_events.valuenum.astype(float)) > 1e-6]

,subject_id,hadm_id,specimen_id,itemid,value,valuenum,valueuom,ref_range_lower,ref_range_upper,comments,flag


In [571]:
# Fill missing valuenum if value is available
mask = lab_events.value.notna() & lab_events.valuenum.isna()
lab_events.loc[mask, 'valuenum'] = lab_events.loc[mask, 'value'].str.replace('<|>', '')

# Fill missing valuenum for glomerular flow rate (GFR) test
mask = lab_events.itemid==50920
gfr_table = lab_events.loc[mask, 'comments'].str.extract('>(\d+)|= (\d+)|between (\d+)')
lab_events.loc[mask, 'valuenum'] = gfr_table[0].fillna(gfr_table[1]).fillna(gfr_table[2])
lab_events.loc[mask, 'ref_range_lower'] = 60
lab_events.loc[mask, 'ref_range_upper'] = 100

# Fill missing valuenum for tests where value is reported in comments
mask = lab_events.value.isna() & lab_events.valuenum.isna()
lab_events.loc[mask, 'valuenum'] = lab_events.loc[mask, 'comments'].str.extract('(\d+.\d+)', expand=False)

# Fix missing flags for abnormal values
lab_events.valuenum = lab_events.valuenum.astype(float)
mask = (lab_events.valuenum < lab_events.ref_range_lower) | (lab_events.valuenum > lab_events.ref_range_upper)
lab_events.loc[mask, 'flag'] = 'abnormal'

In [255]:
# # Save for debugging !!!
# trouble = lab_events[lab_events.value.isna() & lab_events.valuenum.isna()].itemid.unique()
# print(d_lab_events[d_lab_events.itemid.isin(trouble)])
# lab_events[(lab_events.itemid==50885) & lab_events.valuenum.isna()].head()

,itemid,label,fluid,category,loinc_code
229,50885,"Bilirubin, Total",Blood,Chemistry,1975-2
634,50920,Estimated GFR (MDRD equation),Blood,Chemistry,NaN
1283,51275,PTT,Blood,Hematology,5898-2
1494,51003,Troponin T,Blood,Chemistry,6598-7
1598,51301,White Blood Cells,Blood,Hematology,804-5


## 3.C. Calculate aggregate test values

# --------------------------------- STOP ---------------------------------

# Archive

In [ ]:
def how_many_patients(has_words, does_not_have):
    patient_mask = [False] * len(diagnoses)
    for word in has_words:
        patient_mask = np.logical_or(patient_mask, diagnoses['long_title'].str.contains(word))
        print(word)
        print(diagnoses[patient_mask].subject_id.nunique())
    for word in does_not_have:
        patient_mask = np.logical_and(patient_mask, ~diagnoses['long_title'].str.contains(word))
        print(word)
        print(diagnoses[patient_mask].subject_id.nunique())
    num_patients = diagnoses[patient_mask].subject_id.nunique()
    return num_patients, num_patients/diagnoses.subject_id.nunique()

In [ ]:
how_many_patients(['heart failure'], [])

In [ ]:
how_many_patients(['heart failure'], ['kidney'])

In [ ]:
def compare_subjects(series1, series2):
    series1 = series1.values
    series2 = series2.values
#     missing1 = []
#     for s in series2:
#         if s not in series1:
#             missing1.append(s)
    missing2 = []
    for s in series1:
        if s not in series2:
            missing2.append(s)
#     print("There are", len(missing1), " entires in series 2 missing in series 1")
    print(len(missing2), "of the patients in series 1 are not in series 2")
    return missing2

In [ ]:
diag_missing = compare_subjects(adm_list, diag_list)

In [ ]:
drg_missing = compare_subjects(diag_list, drg_list)